# MyModule -> Foundations

In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import os
import gzip
import numpy as np
from itertools import islice
import torch
from torch import tensor
from matplotlib import pyplot as plt
from fastcore.test import test_eq, test_close
from numba import njit
import pdb
from torch import nn
import torch.nn.functional as F

def load_mnist(path, kind='train'):

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

# labels from fashion mnist dataset
labels = ['T-shirt/top', 
          'Trouser',
          'Pullover',
          'Dress',
          'Coat',
          'Sandal',
          'Shirt',
          'Sneaker',
          'Bag',
          'Ankle boot'
         ]

x_train, y_train = load_mnist('../../data/fashion_mnist', kind='train')
x_valid, y_valid = load_mnist('../../data/fashion_mnist', kind='t10k')

x_train, x_valid = x_train.astype('float32') / 255., x_valid.astype('float32') / 255.
x_train, x_valid, y_train, y_valid = map(tensor, (x_train, x_valid, y_train, y_valid))

## MyModule

In [ ]:
class MyModule:
    
    def __init__(self):
        self._module = {}
    
    def __setattr__(self, k, v):
        if  k[0] != '_': self._module[k] = v
        super().__setattr__(k, v)
    
    def __call__(self, x):
        return self.forward(x)
    
    def forward(self, x):
        raise Exception("Not implemented")
        
    def __repr__(self):
        return f'{self._module}'
    
    def parameters(self):
        for l in self._module.values(): yield from l.parameters()

In [ ]:
m, nh, c = 784, 50, 10

In [ ]:
class Model(MyModule):
    def __init__(self, m, nh, c):
        super().__init__()
        self.l1 = nn.Linear(m, nh)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(nh, c)
    
    def forward(self, x):
        return self.l2(self.relu(self.l1(x)))

In [ ]:
model = Model(m, nh, c)

In [ ]:
model

{'l1': Linear(in_features=784, out_features=50, bias=True), 'relu': ReLU(), 'l2': Linear(in_features=50, out_features=10, bias=True)}

In [ ]:
for p in model.parameters():
    print(p.shape)

torch.Size([50, 784])
torch.Size([50])
torch.Size([10, 50])
torch.Size([10])


In [ ]:
model(x_valid).shape

torch.Size([10000, 10])

In [ ]:
class Model(nn.Module):
    def __init__(self, m, nh, c):
        super().__init__()
        self.l1 = nn.Linear(m, nh)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(nh, c)
    
    def forward(self, x):
        return self.l2(self.relu(self.l1(x)))

In [ ]:
model = Model(m, nh, c)

In [ ]:
model

Model(
  (l1): Linear(in_features=784, out_features=50, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=50, out_features=10, bias=True)
)

In [ ]:
for p in model.parameters():
    print(p.shape)

torch.Size([50, 784])
torch.Size([50])
torch.Size([10, 50])
torch.Size([10])


In [ ]:
model(x_valid).shape

torch.Size([10000, 10])

## Register Modules

In [ ]:
layers = [nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, c)]

In [ ]:
from functools import reduce

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = layers
        for i, l in enumerate(self.layers): 
            self.register_module(f'layer_{i}', l)
    
    def forward(self, x):
        return reduce(lambda x, l: l(x), self.layers, x)

In [ ]:
model = Model(); model

Model(
  (layer_0): Linear(in_features=784, out_features=50, bias=True)
  (layer_1): ReLU()
  (layer_2): Linear(in_features=50, out_features=10, bias=True)
)

In [ ]:
for p in model.parameters():
    print(p.shape)

torch.Size([50, 784])
torch.Size([50])
torch.Size([10, 50])
torch.Size([10])


In [ ]:
list(model.named_children())

[('layer_0', Linear(in_features=784, out_features=50, bias=True)),
 ('layer_1', ReLU()),
 ('layer_2', Linear(in_features=50, out_features=10, bias=True))]

In [ ]:
model(x_valid).shape

torch.Size([10000, 10])

## `nn.ModuleList`

In [ ]:
class Model(nn.Module):
    def __init__(self, m, nh, c):
        super().__init__()
        self.layers = nn.ModuleList([nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, c)])
    
    def forward(self, x):
        for l in self.layers: x = l(x)
        return x

In [ ]:
model = Model(m, nh, c); model

Model(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [ ]:
list(model.named_children())

[('layers',
  ModuleList(
    (0): Linear(in_features=784, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  ))]

In [ ]:
model(x_valid).shape

torch.Size([10000, 10])

## `nn.Sequential`

In [ ]:
model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, c))

In [ ]:
model

Sequential(
  (0): Linear(in_features=784, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=10, bias=True)
)

In [ ]:
model(x_valid).shape

torch.Size([10000, 10])

In [ ]:
list(model.named_children())

[('0', Linear(in_features=784, out_features=50, bias=True)),
 ('1', ReLU()),
 ('2', Linear(in_features=50, out_features=10, bias=True))]

## Training 

In [ ]:
def accuracy(preds, act):
    return (preds.argmax(dim=-1) == act).float().mean()

In [ ]:
def report(loss, preds, act):
    print(f"loss: {loss}, accuracy: {accuracy(preds, act)}")

In [ ]:
epochs = 3
bs = 64
lr = 5e-3
loss_func = F.cross_entropy

In [ ]:
def train(epochs, lr):
    for epoch in range(epochs):
        for i in range(0, len(x_train), bs):
            s = slice(i, i + bs)
            xb, yb = x_train[s], y_train[s]
            preds = model(xb)
            loss = loss_func(preds, yb)
            loss.backward()
            with torch.no_grad():
                for p in model.parameters(): p -= p.grad * lr
                model.zero_grad()
        report(loss.item(), preds, yb)

In [ ]:
train(epochs, lr)

loss: 1.0422121286392212, accuracy: 0.65625
loss: 0.8457322716712952, accuracy: 0.71875
loss: 0.7601563930511475, accuracy: 0.71875


## opt

In [ ]:
class Opt:
    def __init__(self, params, lr=0.001):
        self.params = list(params)
        self.lr = lr
    
    def step(self):
        with torch.no_grad():
            for p in self.params: p -= p.grad * lr
    
    def zero_grad(self):
        for p in self.params: p.grad.data.zero_()

In [ ]:
opt = Opt(model.parameters())

In [ ]:
for epoch in range(epochs):
    for i in range(0, len(x_train), bs):
        s = slice(i, i + bs)
        xb, yb = x_train[s], y_train[s]
        preds = model(xb)
        loss = loss_func(preds, yb)
        loss.backward()
        opt.step()
        opt.zero_grad()
    report(loss.item(), preds, yb)

loss: 0.7086510062217712, accuracy: 0.75
loss: 0.6750910878181458, accuracy: 0.8125
loss: 0.651551365852356, accuracy: 0.8125


In [ ]:
from torch import optim

In [ ]:
def get_model():
    model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, c))
    return model, optim.SGD(model.parameters(), lr=5e-3)

In [ ]:
model, opt = get_model()
loss_func(model(x_train), y_train)

tensor(2.3087, grad_fn=<NllLossBackward0>)

In [ ]:
for epoch in range(epochs):
    for i in range(0, len(x_train), bs):
        s = slice(i, i + bs)
        xb, yb = x_train[s], y_train[s]
        preds = model(xb)
        loss = loss_func(preds, yb)
        loss.backward()
        opt.step()
        opt.zero_grad()
    report(loss.item(), preds, yb)

loss: 1.1073848009109497, accuracy: 0.6875
loss: 0.9047041535377502, accuracy: 0.71875
loss: 0.8060352802276611, accuracy: 0.71875


## Dataset

In [ ]:
class Dataset:
    def __init__(self, x, y):
        self.x, self.y = x, y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [ ]:
train_ds = Dataset(x_train, y_train)
valid_ds = Dataset(x_valid, y_valid)

In [ ]:
xb, yb = train_ds[:5]

In [ ]:
assert len(train_ds) == len(x_train)
assert xb.shape == (5, 28*28)
assert yb.shape == (5,)

In [ ]:
model, opt = get_model()

In [ ]:
for epoch in range(epochs):
    for i in range(0, len(train_ds), bs):
        xb, yb = train_ds[i: i+bs]
        preds = model(xb)
        loss = loss_func(preds, yb)
        loss.backward()
        opt.step()
        opt.zero_grad()
    report(loss, preds, yb)

loss: 1.0646525621414185, accuracy: 0.65625
loss: 0.8715953826904297, accuracy: 0.6875
loss: 0.7849368453025818, accuracy: 0.71875


## DataLoader

In [ ]:
class DataLoader:
    def __init__(self, ds, bs):
        self.ds = ds
        self.bs = bs
    
    def __iter__(self):
        for i in range(0, len(self.ds), self.bs): yield self.ds[i: i + bs]

In [ ]:
train_dl = DataLoader(train_ds, bs)

In [ ]:
for epoch in range(epochs):
    for xb, yb in train_dl:
        preds = model(xb)
        loss = loss_func(preds, yb)
        loss.backward()
        opt.step()
        opt.zero_grad()
    report(loss, preds, yb)

loss: 0.7310202121734619, accuracy: 0.75
loss: 0.6964010000228882, accuracy: 0.8125
loss: 0.6729718446731567, accuracy: 0.8125


## Random Sampling

### Sampler

In [ ]:
import random; random.shuffle([1, 2, 3, 4, 5])

In [ ]:
class Sampler:
    def __init__(self, ds, shuffle=False):
        self.n = len(ds)
        self.shuffle = shuffle
    
    def __iter__(self):
        nums = list(range(self.n))
        if self.shuffle: random.shuffle(nums)
        return iter(nums)

In [ ]:
ss = Sampler(train_ds)

In [ ]:
it = iter(ss)

In [ ]:
for i in range(5): print(next(it))

0
1
2
3
4


In [ ]:
list(islice(it, 5))

[5, 6, 7, 8, 9]

In [ ]:
random_sampler = Sampler(train_ds, shuffle=True)

In [ ]:
it = iter(random_sampler)

In [ ]:
for i in range(5): print(next(it))

44940
47754
46503
25176
18910


In [ ]:
list(islice(it, 5))

[43146, 53076, 59023, 12286, 30258]

## Batch Sampler

In [ ]:
import fastcore.all as fc

In [ ]:
class BatchSampler:
    def __init__(self, sampler, bs, drop_last=False):
        fc.store_attr()
    
    def __iter__(self):
        yield from fc.chunked(self.sampler, self.bs, self.drop_last)

In [ ]:
batch_sampler = BatchSampler(Sampler(train_ds, shuffle=True), 4)

In [ ]:
list(islice(batch_sampler, 5))

[[10169, 1359, 40945, 44900],
 [24941, 15785, 48809, 2895],
 [26045, 4747, 35952, 12967],
 [37500, 39354, 7712, 54674],
 [9018, 36599, 14359, 49447]]

In [ ]:
class DataLoader:
    def __init__(self, ds, batch_sampler):
        fc.store_attr()
    
    def __iter__(self):
        yield from (self.ds[b] for b in self.batch_sampler)

In [ ]:
train_dl = DataLoader(train_ds, BatchSampler(Sampler(train_ds, shuffle=True), bs))
model, opt = get_model()

In [ ]:
def fit():
    for epoch in range(epochs):
        for xb, yb in train_dl:
            preds = model(xb)
            loss = loss_func(preds, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        report(loss, preds, yb)

In [ ]:
def collate(b):
    xb, yb = zip(*b)
    return torch.stack(xb), torch.stack(yb)

In [ ]:
class DataLoader:
    def __init__(self, ds, batch_sampler, collate_fn=collate):
        fc.store_attr()
    
    def __iter__(self):
        yield from (self.collate_fn(self.ds[i] for i in b) for b in self.batch_sampler)

In [ ]:
train_dl = DataLoader(train_ds, BatchSampler(Sampler(train_ds, shuffle=True), bs))

In [ ]:
xb, yb = next(iter(train_dl))
xb.shape, yb.shape

(torch.Size([64, 784]), torch.Size([64]))

## Multiprocessing DataLoader

In [ ]:
import torch.multiprocessing as mp

In [ ]:
def square(x):
    return x ** 2

In [ ]:
with mp.Pool(2) as process:
    res = process.map(square, torch.tensor([2, 4, 5, 6, 7, 8]))

In [ ]:
train_ds[[2, 3]]

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([0, 3], dtype=torch.uint8))

In [ ]:
train_ds.__getitem__([2, 3])

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([0, 3], dtype=torch.uint8))

In [ ]:
class DataLoader:
    def __init__(self, ds, batch_sampler, n_workers=1):
        fc.store_attr()
    
    def __iter__(self):
        with mp.Pool(self.n_workers) as process:
            yield from process.map(self.ds.__getitem__, self.batch_sampler)

In [ ]:
train_dl = DataLoader(train_ds, BatchSampler(Sampler(train_ds, shuffle=True), bs), 2)

In [ ]:
it = iter(train_dl)
xb, yb = next(it)
xb.shape, yb.shape

(torch.Size([64, 784]), torch.Size([64]))

## PyTorch DataLoader

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, BatchSampler, SequentialSampler

In [ ]:
train_samp = BatchSampler(RandomSampler(train_ds), batch_size=bs, drop_last=False)
valid_samp = BatchSampler(SequentialSampler(valid_ds), batch_size=bs*2, drop_last=False)

In [ ]:
train_dl = DataLoader(train_ds, batch_sampler=train_samp)
valid_dl = DataLoader(valid_ds, batch_sampler=valid_samp)

In [ ]:
it = iter(train_dl)
xb, yb = next(it)
xb.shape, yb.shape

(torch.Size([64, 784]), torch.Size([64]))

In [ ]:
it = iter(valid_dl)
xb, yb = next(it)
xb.shape, yb.shape

(torch.Size([128, 784]), torch.Size([128]))

In [ ]:
train_dl = DataLoader(train_ds, batch_size=bs, sampler=RandomSampler(train_ds))
valid_dl = DataLoader(valid_ds, batch_size=bs*2, sampler=SequentialSampler(valid_ds))

In [ ]:
it = iter(train_dl)
xb, yb = next(it)
xb.shape, yb.shape

(torch.Size([64, 784]), torch.Size([64]))

In [ ]:
xb, yb = next(iter(valid_dl))
xb.shape, yb.shape

(torch.Size([128, 784]), torch.Size([128]))

In [ ]:
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=bs*2, shuffle=False)

## Validation

In [ ]:
for epoch in range(epochs):
    for xb, yb in train_dl:
        preds = model(xb)
        loss = loss_func(preds, yb)
        loss.backward()
        opt.step()
        opt.zero_grad()
    
    loss_sum = 0
    acc_sum = 0
    count = 0
    for xb, yb in valid_dl:
        n = len(xb)
        count += n
        preds = model(xb)
        loss = loss_func(preds, yb)
        loss_sum += loss.item() * n
        acc_sum += accuracy(preds, yb).item() * n
    
    total_loss = loss_sum / count
    total_acc = acc_sum / count
    print(total_loss, total_acc)

0.9977972864151001 0.6638
0.7845826868057251 0.713
0.6950922035694123 0.7624


## HuggingFace Dataset

In [ ]:
from datasets import load_dataset
from operator import itemgetter
import torchvision.transforms.functional as TF

In [ ]:
ds = load_dataset("fashion_mnist")
ds

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [ ]:
ig = itemgetter('train', 'test')
train_ds, valid_ds = ig(ds)

In [ ]:
x, y = train_ds.features
x, y

('image', 'label')

In [ ]:
featy = train_ds.features[y]; featy

ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)

In [ ]:
train_ds[0][x]

In [ ]:
featy.int2str(train_ds[0][y])

'Ankle boot'

In [ ]:
itemgetter(x, y)(train_ds[0: 5])

([<PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>],
 [9, 0, 0, 3, 0])

In [ ]:
def collate(b):
    xs = torch.stack([torch.flatten(TF.to_tensor(ele[x])) for ele in b])
    ys = torch.tensor([ele[y] for ele in b])
    return xs, ys

In [ ]:
train_dl = DataLoader(train_ds, shuffle=True, batch_size=bs, collate_fn=collate)

In [ ]:
xb, yb = next(iter(train_dl))
xb.shape, yb.shape

(torch.Size([64, 784]), torch.Size([64]))

In [ ]:
def transform(b):
    b[x] = [torch.flatten(TF.to_tensor(img)) for img in b[x]]
    return b

In [ ]:
train_ds = train_ds.with_transform(transform); train_ds[0][x].shape

torch.Size([784])

In [ ]:
from torch.utils.data import default_collate

In [ ]:
default_collate([dict(a=1, b=2), dict(a=2, b=3)])

{'a': tensor([1, 2]), 'b': tensor([2, 3])}

In [ ]:
def collate(b):
    b = default_collate(b)
    xs, ys = itemgetter(x, y)(b)
    return xs, ys

In [ ]:
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True, collate_fn=collate)
valid_dl = DataLoader(valid_ds, batch_size=bs*2, shuffle=False, collate_fn=collate)

In [ ]:
it = iter(train_dl)
xb, yb = next(it)
xb.shape, yb.shape

(torch.Size([64, 784]), torch.Size([64]))

## Callback

In [ ]:
import ipywidgets as widgets
import time

In [ ]:
button = widgets.Button(description="Click Me!");
button

Button(description='Click Me!', style=ButtonStyle())

In [ ]:
button.on_click(lambda x: print("Hi Anubhav!"))

In [ ]:
def work(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        if cb: cb(i)
        time.sleep(1)
    return res

In [ ]:
def show_progress(epoch):
    print(f"Hi! we are at the {epoch}")

In [ ]:
work(show_progress)

Hi! we are at the 0
Hi! we are at the 1
Hi! we are at the 2
Hi! we are at the 3
Hi! we are at the 4


30

In [ ]:
work(lambda o: print(f"Hello! We are the iteration {o}"))

Hello! We are the iteration 0
Hello! We are the iteration 1
Hello! We are the iteration 2
Hello! We are the iteration 3
Hello! We are the iteration 4


30

In [ ]:
def show_progress(exclamation, epoch):
    print(f"{exclamation} We are at the {epoch}")

In [ ]:
work(lambda o: show_progress("Howdy!", o))

Howdy! We are at the 0
Howdy! We are at the 1
Howdy! We are at the 2
Howdy! We are at the 3
Howdy! We are at the 4


30

In [ ]:
from functools import partial

In [ ]:
work(partial(show_progress, "Namaste!"))

Namaste! We are at the 0
Namaste! We are at the 1
Namaste! We are at the 2
Namaste! We are at the 3
Namaste! We are at the 4


30

In [ ]:
def g(*args, **kwargs):
    print(args)
    print(kwargs)

In [ ]:
g(1, 2, k=3)

(1, 2)
{'k': 3}


In [ ]:
def g(a, b, c=1):
    print(a, b, c)

In [ ]:
args = [1, 2]
kwargs = {'c': 3}

In [ ]:
g(*args, **kwargs)

1 2 3


In [ ]:
class PrintProgressCallback:
    def __init__(self): pass
    def before_calc(self, epoch, val, *args, **kwargs): print(f'Before calculation {epoch}: {val}')
    def after_calc(self, epoch, val, *args, **kwargs): print(f'After calculation {epoch}: {val}')

In [ ]:
def work1(cb=None):
    res = 0
    for i in range(5):
        cb.before_calc(i, res)
        res += i*i
        cb.after_calc(i, res)
    return res

In [ ]:
work1(PrintProgressCallback())

Before calculation 0: 0
After calculation 0: 0
Before calculation 1: 0
After calculation 1: 1
Before calculation 2: 1
After calculation 2: 5
Before calculation 3: 5
After calculation 3: 14
Before calculation 4: 14
After calculation 4: 30


30

## `__dunder__`

In [ ]:
class SloppyMultiplier:
    def __init__(self, o):
        self.o = o
    
    def __mul__(self, b):
        return SloppyMultiplier(self.o * b.o * 0.1)
    
    def __repr__(self):
        return str(self.o)

In [ ]:
a = SloppyMultiplier(1)
b = SloppyMultiplier(2)
a * b

0.2

In [ ]:
class B:
    a, b = 1, 2
    def __getattr__(self, k):
        if k[0] == '_': raise Exception("It is private method")
        return f"Hello from {k}"

In [ ]:
b = B()

In [ ]:
b.a

1

In [ ]:
b.b

2

In [ ]:
b.Anubhav

'Hello from Anubhav'